In [14]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import glob

- import dataset
- import phobert tokenizer and vectorizer
- create dataloader
- define encoder
- define attention
- define decoder
- define seq2seq

WIP:
+ train loop
+ validate

In [59]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:
# input = 'Ngày 27/3 , Cơ_quan Cảnh_sát điều_tra Công_an TP. Hưng_Yên , tỉnh Hưng_Yên cho biết , đơn_vị vừa ra quyết_định khởi_tố vụ án , khởi_tố bị_can đối_với đối_tượng Mai_Văn_Thương ( SN 1989 , trú tại đội 11 , thôn An_Chiểu 1 , xã Liên_Phương , TP. Hưng_Yên ) để điều_tra về hành_vi trộm_cắp tài_sản. Theo tài_liệu điều_tra của cơ_quan công_an , vào_khoảng 7h30 ngày 13/3 , lợi_dụng gia_đình ông Mai_Văn_Thịnh ( chú ruột đối_tượng Thương ) ở cạnh nhà đi vắng , đối_tượng này đã đạp gãy chấn_song cửa_sổ , đột_nhập vào nhà ông Thịnh trộm_cắp 121kg thóc mang bán cho người cùng thôn lấy 700.000 đ . Không dừng lại , sau đó đối_tượng tiếp_tục quay lại lục_soát tủ nhà ông Thịnh trộm_cắp 8.500.000 đ tiền_mặt ( ông Thịnh để dưới đáy tủ ) , rồi dùng số tiền trên để đi mua ma_tuý về sử_dụng và tiêu_xài hết 6.080.000 đ . Đến ngày 15/3 , đối_tượng Thương đã đến Cơ_quan điều_tra Công_an TP. Hưng_Yên tự_thú và khai nhận toàn_bộ hành_vi phạm_tội của mình , đồng_thời giao_nộp cho cơ_quan công_an 3.120.000 đ . Hiện Công_an TP. Hưng_Yên đã thu_giữ toàn_bộ 121kg thóc đối_tượng đã trộm_cắp để trao_trả cho gia_đình ông Thịnh . Được biết Thương là đối_tượng nghiện ma_tuý từ nhiều năm nay , đã có 1 tiền_án về tội Tàng_trữ trái_phép chất ma_tuý bị TAND tỉnh Hưng_Yên xử_phạt 2 năm 3 tháng tù_giam. Ra tù năm 2016, đối_tượng này tiếp_tục có hành_vi cố_ý gây thương_tích , bị Công_an TP. Hưng_Yên ra quyết_định xử_phạt 2,5 triệu đồng . Vụ án đang được Công_an TP. Hưng_Yên hoàn_thiện hồ_sơ để xử_lý Mai_Văn_Thương theo quy_định của pháp_luật .'
# target = 'Với bản_tính ham chơi , lười làm , có nhiều tiền_án tiền_sự , lại nghiện ma_tuý , Thương đã đột_nhập vào nhà chú ruột để trộm hơn 1 tạ thóc và hơn 8 triệu đồng mang đi tiêu_xài .'

In [2]:
#!git clone 'https://github.com/ThanhChinhBK/vietnews'

Cloning into 'vietnews'...
remote: Enumerating objects: 143827, done.
remote: Counting objects: 100% (143827/143827), done.
remote: Compressing objects: 100% (143815/143815), done.
remote: Total 143827 (delta 11), reused 143827 (delta 11), pack-reused 0
Receiving objects: 100% (143827/143827), 194.68 MiB | 18.30 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Checking out files: 100% (150704/150704), done.


In [18]:
!pip install transformers

     |████████████████████████████████| 1.3MB 9.8MB/s 
     |████████████████████████████████| 1.1MB 33.2MB/s 
     |████████████████████████████████| 890kB 45.1MB/s 
     |████████████████████████████████| 2.9MB 46.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=1ad2d5b5c92fdfc7ad37f931ca7ae690fb0257e8b4530bdf04e989a6da69fe61
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [1]:
!wget -O phobert.tar.gz https://public.vinai.io/PhoBERT_base_transformers.tar.gz
!tar -zxvf phobert.tar.gz

--2020-11-18 13:30:10--  https://public.vinai.io/PhoBERT_base_transformers.tar.gz
Resolving public.vinai.io (public.vinai.io)... 99.86.34.107, 99.86.34.9, 99.86.34.108, ...
Connecting to public.vinai.io (public.vinai.io)|99.86.34.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322405979 (307M) [application/x-tar]
Saving to: ‘phobert.tar.gz’

phobert.tar.gz      100%[===================>] 307.47M  14.3MB/s    in 23s     

2020-11-18 13:30:34 (13.5 MB/s) - ‘phobert.tar.gz’ saved [322405979/322405979]

PhoBERT_base_transformers/
PhoBERT_base_transformers/config.json
PhoBERT_base_transformers/bpe.codes
PhoBERT_base_transformers/model.bin
PhoBERT_base_transformers/dict.txt


In [25]:
from transformers import AutoModel, AutoTokenizer, AutoConfig

config = AutoConfig.from_pretrained('/content/PhoBERT_base_transformers/config.json')
phobert = AutoModel.from_pretrained('vinai/phobert-base', config = config)
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base')

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing BertModel: ['roberta.embeddings.position_ids', 'roberta.embeddings.word_embeddings.weight', 'roberta.embeddings.position_embeddings.weight', 'roberta.embeddings.token_type_embeddings.weight', 'roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.0.attention.self.query.weight', 'roberta.encoder.layer.0.attention.self.query.bias', 'roberta.encoder.layer.0.attention.self.key.weight', 'roberta.encoder.layer.0.attention.self.key.bias', 'roberta.encoder.layer.0.attention.self.value.weight', 'roberta.encoder.layer.0.attention.self.value.bias', 'roberta.encoder.layer.0.attention.output.dense.weight', 'roberta.encoder.layer.0.attention.output.dense.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.intermediate.dense.weight', 'roberta.encoder.layer.0.intermediat

In [6]:
class VietDataset(Dataset):
    def __init__(self, data_path):
        self.files_path = glob.glob(f'{data_path}/*')

    def __len__(self):
        return len(self.files_path)
        
    def __getitem__(self, idx):
        file_path = self.files_path[idx]

        with open(file_path, 'r') as f:
            file_content = f.readlines()
            target = file_content[2]
            text = ' '.join(file_content[3:]).replace('\n', '')

            input_ids = torch.tensor([tokenizer.encode(text)])
            target_ids = torch.tensor([tokenizer.encode(target)])

            with torch.no_grad():
              text = phobert(input_ids)
              target = phobert(target_ids)



        return text, target

In [7]:
def make_dataloader(dataset):
    dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)
    return dataloader

In [26]:
train_iterator = VietDataset('/content/vietnews/data/train_tokenized')

In [36]:
for i in range(5):
  sample1, sample2 = train_iterator[i]
  print(sample1[0].shape)

torch.Size([1, 518, 768])
torch.Size([1, 487, 768])
torch.Size([1, 352, 768])
torch.Size([1, 687, 768])
torch.Size([1, 286, 768])


In [80]:
# input_ids = torch.tensor([tokenizer.encode(input)])
# target_ids = torch.tensor([tokenizer.encode(target)])

# with torch.no_grad():
#   x = phobert(input_ids)
#   y = phobert(target_ids)

# x[0].squeeze().size()
# x[0].size()
# print(x[0].shape)
# # print(x[1].shape)
# print(y[0].shape)
# for i in range(0,9):
#   print(features[0][0,i,:5])
# print(' aaa ')
# print(features[1][0,:5])
# print(tokenizer.tokenize(input))

torch.Size([1, 295, 768])

In [82]:
class EncoderBiLSTM(nn.Module):
  def __init__(self, input_dim, output_dim, enc_hidden_dim, dec_hidden_dim, num_layers, dropout):
    super().__init__()
    self.dropout = nn.Dropout(dropout)
    self.linear = nn.Linear(input_dim, output_dim)
    self.lstm = nn.LSTM(input_size = output_dim,
                        hidden_size = enc_hidden_dim,
                        num_layers = num_layers,
                        dropout = dropout,
                        bidirectional = True)
    self.out = nn.Linear(enc_hidden_dim*2, dec_hidden_dim)

  def forward(self, input):
    # input: (sent len, batch size)
    output = self.dropout(self.linear(input))
    output, (h_n, h_c) = self.lstm(output)   # output: (sent len, batch size, hidden dim*num directions), h_n: (num layers*num directions, batch size, hidden dim), h_c: (num layers*num directions, batch size, hidden dim)
    hidden = self.out(torch.cat((h_n[-2,::], h_n[-1,:,:]), dim=1))   # hidden: (sent len, batch size, num directions*hidden dim)
    # h_n[-2,:,:] last states of forward pass, h_n[-1,:,:] last states of backward pass
    hidden = torch.tanh(hidden)   # hidden: (batch size, dec_hidden_dim)

    return output, hidden

In [73]:
class Attention(nn.Module):
  def __init__(self, enc_hidden_dim, dec_hidden_dim):
    super().__init__()
    self.attn = nn.Linear(enc_hidden_dim*2 + dec_hidden_dim, dec_hidden_dim)
    self.v = nn.Linear(dec_hidden_dim, 1, bias = False)

  def forward(self, hidden, encoder_outputs):
    batch_size = encoder_outputs.shape[1]
    src_len = encoder_outputs.shape[0]
    hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
    encoder_outputs = encoder_outputs.permute(1, 0, 2)
    energy = torch.cat((hidden, encoder_outputs), sim=2)
    energy = torch.tanh(self.attn(energy))
    attention = self.v(energy).squeeze(2)

    return F.softmax(attention, dim=1)


In [74]:
class DecoderLSTM(nn.Module):
  def __init__(self, output_dim, embedding_dim, enc_hidden_dim, dec_hidden_dim, dropout, attention):
      super().__init__()
      self.output_dim = output_dim
      self.attention = attention
      self.embedding = nn.Embedding(output_dim, embedding_dim)
      self.lstm = nn.LSTM(enc_hidden_dim*2 + embedding_dim, dec_hidden_dim)
      self.out = nn.Linear(enc_hidden_dim*2 + dec_hidden_dim + embedding_dim, output_dim)
      self.dropout = nn.Dropout(dropout)

  def forward(self, input, hidden, enc_out):
      input = input.unsqueeze(0)
      embedded = self.dropout(self.embedding(input))
      
      attn = self.attention(hidden, enc_out)
      attn = attn.unsqueeze(1)
      anc_out = enc_out.permute(1, 0, 2)
      weighted = torch.bmm(attn, enc_out)
      weighted = weighted.permute(1, 0, 2)
      
      lstm_input = torch.cat((embedded, weighted), dim=2)
      output, hidden = self.lstm(lstm_input, hidden.unsqueeze(0))
      
      assert (output == hidden).all()

      embedded = embedded.squeeze(0)
      output = output.squeeze(0)
      weighted.squeeze(0)

      pred = torch.cat((output, weighted, embedded), dim=1)
      pred = self.out(pred)

      return pred, hidden.squeeze(0)

In [75]:
class seq2seq(nn.Module):
  def __init__(self, encoder, decoder, device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device

  def forward(self, src, trg, teacher_forcing_ratio = 0.5):
    batch_size = src.shape[1]
    trg_len = tfg.shape[0]
    trg_vocab_size = self.decoder.output_dim

    outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

    encoder_outputs, hidden = self.encoder(src)

    input = trg[0,:]

    for t in range(1, trg_len):
      output, hidden = self.decoder(input, hidden, encoder_outputs)
      outputs[t] = output
      teacher_force = random.random() < teacher_forcing_ratio
      top1 = output.argmax(1)
      input = trg[t] if teacher_force else top1

    return outputs

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

In [81]:
input_dim = 768
output_dim = 768
enc_hidden_dim = 512
dec_hidden_dim = 512
enc_dropout = 0.5
dec_dropout = 0.5
enc_layers = 2

attn = Attention(enc_hidden_dim, dec_hidden_dim)
enc = EncoderBiLSTM(input_dim, output_dim, enc_hidden_dim, dec_hidden_dim, enc_layers, enc_dropout)
dec = DecoderLSTM(output_dim, output_dim, enc_hidden_dim, dec_hidden_dim, dec_dropout, attn)

model = seq2seq(enc, dec, device).to(device)

In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
  model.train()
  epoch_loss = 0
  for i, batch in enumerate(iterator):
    src = text
    trg = target

    optimizer.zero_grad()

    output = model(src, trg)

    output_dim = output.shape[-1]

    output = output[1:].view(-1, output_dim)
    trg = trg[1:].view(-1)

    loss = criterion(output, trg)
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

    optimizer.step()

    epoch_loss += loss.item()

  return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
  model.eval()
  epoch_loss = 0

  with torch.no_grad():
    for i, batch in enumerate(iterator):
      src = text
      trg = target

      output = model(src, trg, 0)
      output_dim = output.shape[-1]
      output = output[1:].view(-1, output_dim)
      trg = trg[1:].view(-1)

      loss = criterion(output, trg)

      epoch_loss += loss.item()

  return epoch_loss / len(iterator)


In [ ]:
# train_iterator = make_dataloader('/content/vietnews/data/train_tokenized')
# valid_iterator = make_dataloader('/content/vietnews/data/val_tokenized')
# test_iterator = make_dataloader('/content/vietnews/data/test_tokenized')

In [ ]:
n_epochs = 10
clip = 1

best_valid_loss = float('inf')

for epoch in range(n_epochs):
  train_loss = train(model, train_iterator, optimizer, criterion, clip)
  valid_loss = evaluate(model, valid_iterator, criterion)

  if valid_loss < best_valid_loss:
    best_valid_loss = best_valid_losstorch.save(model.state_dict(), 'model.pt')

In [ ]:
model.load_state_dict(torch.load('model.pt'))

test_loss = evaluate(model, test_iterator, criterion)
print(test_loss)